In [1]:
!nvidia-smi

Mon Mar 14 12:19:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 6.5 MB 65.3 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
import sys

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Clean_Data.csv')

In [6]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'id', 'dialect', 'text',
       'clean_text'],
      dtype='object')

In [7]:
df.drop(labels=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','id','text'],axis=1,inplace=True)

In [8]:
df = df[['clean_text','dialect']]

In [9]:
df['clean_text']=df['clean_text'].values.astype('U')

In [50]:
df[['dialect','dialect_class']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   dialect        458197 non-null  object
 1   dialect_class  458197 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
df.head()

,clean_text,dialect
0,بالنهاية ينتفض يغير,IQ
1,يعني محسوب البشر حيونه ووحشيه وتطلبون ال...,IQ
2,مبين كلامه خليجي,IQ
3,يسلملي مرورك وروحك الحلوه,IQ
4,وين الغيبه اخ محمد,IQ


In [12]:
encoder = LabelEncoder()
encoder.fit(df['dialect'])
df['dialect_class']=encoder



In [ ]:
tags = 

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
num_classes = int((len(set(df['dialect_class']))))
print((len(set(df['dialect_class']))))

18


In [15]:
df.dropna()

,clean_text,dialect,dialect_class
0,بالنهاية ينتفض يغير,IQ,4
1,يعني محسوب البشر حيونه ووحشيه وتطلبون ال...,IQ,4
2,مبين كلامه خليجي,IQ,4
3,يسلملي مرورك وروحك الحلوه,IQ,4
4,وين الغيبه اخ محمد,IQ,4
...,...,...,...
458192,مبسوطين منك اللي باسطانا,BH,1
458193,والله ماينده ابش يختي,BH,1
458194,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي...,BH,1
458195,الله يبارك وبالعافيه,BH,1


In [16]:
df

,clean_text,dialect,dialect_class
0,بالنهاية ينتفض يغير,IQ,4
1,يعني محسوب البشر حيونه ووحشيه وتطلبون ال...,IQ,4
2,مبين كلامه خليجي,IQ,4
3,يسلملي مرورك وروحك الحلوه,IQ,4
4,وين الغيبه اخ محمد,IQ,4
...,...,...,...
458192,مبسوطين منك اللي باسطانا,BH,1
458193,والله ماينده ابش يختي,BH,1
458194,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي...,BH,1
458195,الله يبارك وبالعافيه,BH,1


In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
token = tokenizer.encode_plus(
    df['clean_text'].iloc[0],  
    max_length = 256,
    truncation=True,
    padding = 'max_length',
    add_special_tokens = True,
    return_tensors = 'tf'


)

In [19]:
X_input_ids = np.zeros((len(df),256))
X_attn_masks = np.zeros((len(df),256))

In [20]:
X_input_ids.shape

(458197, 256)

In [21]:
def generate_trainig_data(df, ids, masks, tokenizer):
  for i, text in tqdm(enumerate(df['clean_text'])):
    tokenized_text = tokenizer.encode_plus(
      text,  
      max_length = 256,
      truncation=True,
      padding = 'max_length',
      add_special_tokens = True,
      return_tensors = 'tf'
    )
    ids[i, :] = tokenized_text.input_ids
    masks[i, :]=tokenized_text.attention_mask

  return ids, masks 

In [22]:
X_input_ids , X_attn_masks = generate_trainig_data(df,X_input_ids,X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [23]:
X_input_ids.shape

(458197, 256)

In [24]:
labels = np.zeros((len(df),18))
labels.shape

(458197, 18)

In [25]:
labels[np.arange(len(df)),df['dialect_class'].values] = 1

In [26]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [28]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(18,), dtype=tf.float64, name=None))>

In [29]:
def DialectDatasetMapFunction(input_ids, attn_masks, labels):
  return {'input_ids': input_ids, 'attention_mask': attn_masks }, labels

In [30]:
dataset = dataset.map(DialectDatasetMapFunction)

In [31]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(18,), dtype=tf.float64, name=None))>

In [32]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)

In [33]:
p = 0.8
train_size = int((len(df)//16)*p)

In [34]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)


In [35]:
from transformers import TFBertModel

In [36]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [37]:
input_ids = tf.keras.layers.Input(shape=(256,),name='input_ids', dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,),name='attention_mask', dtype='int32')

bert_embds = bert_model.bert(input_ids, attention_mask=attention_masks)[1]
intermediate_layer = tf.keras.layers.Dense(512, activation='relu',name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(18, activation='softmax',name = 'output_layer')(intermediate_layer)


model = tf.keras.Model(inputs = [input_ids, attention_masks],outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [38]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5,decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [39]:
model.compile(optimizer=optim,loss=loss_func,metrics=[acc])

In [40]:
hist = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs = 1

)

22909/22909 [==============================] - 20602s 899ms/step - loss: 0.4106 - accuracy: 0.8416 - val_loss: 9.0894 - val_accuracy: 0.1526


In [60]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [41]:
def prepare_data(input_text, tokenizer):
  token = tokenizer.encode_plus(
    df['clean_text'].iloc[0],  
    max_length = 256,
    truncation=True,
    padding = 'max_length',
    add_special_tokens = True,
    return_tensors = 'tf')
  return {
      'input_ids' : tf.cast(token.input_ids, tf.float64),
      'attention_mask' : tf.cast(token.attention_mask, tf.float64)
  }
    
  

In [42]:
input_text = 'هلا بالغالي في منزلنا'

In [43]:
tokenized_input_text = prepare_data(input_text, tokenizer)

In [44]:
probs = model.predict(tokenized_input_text)

In [45]:
probs

array([[6.97853420e-06, 1.08539325e-05, 3.54672578e-04, 1.22685009e-03,
        6.21776489e-05, 1.72847023e-04, 3.94967943e-01, 9.81383678e-03,
        6.83957769e-04, 1.61176620e-04, 5.88276505e-01, 1.03084475e-03,
        1.24354183e-03, 1.01781345e-03, 3.07898008e-05, 1.75961046e-04,
        5.53427613e-04, 2.09827165e-04]], dtype=float32)

In [57]:
classes = ['AE','BH','DZ','EG','IQ','JO','KW','LB','LY','MA','OM','PL','QA','SA','SD','SY','TN','YE']

In [59]:
output_index = np.argmax(probs[0])
print(classes[output_index])

OM
